In [1]:
%matplotlib qt5
import numpy as np
from numpy import linalg as la
import matplotlib.pylab as plt
from scipy.optimize import fsolve
import math
import statsmodels.api as sm
from scipy.stats import norm
import seaborn as sns
from scipy import stats
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

In [4]:
''' Parameters used for all networks (reciprocal and chain)'''
N =1750#1750#2000
J = 1/np.sqrt(1500)*0.5
### get the current path location and read the data file
import os
strr= os.getcwd()
print(strr)

c = 0.2
# sigma2 = N*c*(1-c)*J**2
g, gamma = 6.8, 1/4.0#1.5,1/1.# 6.8, 1/4.0#1.5,1/1.#
NE = int(N/(1+gamma))
NI = int(gamma*NE)
N  = NE+NI
print(N,NE,NI)
ALPHAE, ALPHAI = NE/N, NI/N
KE, KI = int(c*NE), int(c*NI) ### fixed out-degree
print('number of connected E/I neurons:',KE,KI)
print('non-zero J:',J)
ji,je = g*J,J 

### define the network parameters of the diluted Gaussian matrix 
ge, gi = np.sqrt(je**2*c*(1-c)*N), np.sqrt(ji**2*c*(1-c)*N) 
hat_sigmae, hat_sigmai = np.sqrt(c*(1-c)), np.sqrt(c*(1-c))### standard deviation of the adjacency matrix
sigmae,sigmai = np.sqrt(c*(1-c)*J**2*N), np.sqrt(c*(1-c)*(-g*J)**2*N)### with magnitude of the coupling
JE,JI = je*c*NE, ji*c*NI 
lambda0 = JE-JI 
print('lambda0:',lambda0)
print('ge,gi:',ge,gi)
print('JE,JI:',JE,JI)

### construct the mean matrix


e:\Dropbox\DailyWork\Allen_project\sonets
1750 1400 350
number of connected E/I neurons: 280 70
non-zero J: 0.012909944487358056
lambda0: -2.530349119522179
ge,gi: 0.2160246899469287 1.4689678916391151
JE,JI: 3.6147844564602556 6.145133575982435


In [8]:
# for value in {0.   0.14 0.28 0.42 0.56 0.7  0.84 0.98 1.12 1.26 1.4 }; do
#     ./run_secorder 1000 0.2 $(echo "$value*2" | bc) $value $value 1 1
# done
for seed in {1 7 8 12 13 19 28 31 33 39 40 42 48 50 51 52 55 56 59 65 66 67 69 70 75 76 78 82 83 86 94}; do     mkdir -p "seed_$seed"; done

for seed in {0..36}; do     mkdir -p "seed_$seed"; done

In [5]:
ntau = 10#21#
trials = 30+6
tau_series = np.linspace(0,0.225,ntau)# np.linspace(0.25,0.4,ntau)#
alpha_series = (tau_series*c*(1-c)+c**2)/c**2-1
alpha_series_rec = (tau_series*2*c*(1-c)+c**2)/c**2-1
print(alpha_series)
print(alpha_series_rec)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
[0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8]


In [6]:
tau_series

array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225])

In [10]:
((alpha_series+1)*c**2-c**2)/c/(1-c)

array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225])

In [112]:
# tau_series = np.array([0.025, 0.05 , 0.075, 0.1  ])
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
### normalization for left and right eigenvectors 
norms_for_lvec, norms_for_rvec = np.zeros((trials,ntau+1,2)), np.zeros((trials,ntau+1,2))
### also have the reconstructed left and right eigenvectors 
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
## first eigenvector
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
## second eigenvector
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]  = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left
### compute the square of the random connectivity 
Z2E = N*J**2*hat_sigmae**2*tau_series*ALPHAE-N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE+N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E, DeltaZ2I = Z2E[1]-Z2E[0],Z2I[1]-Z2I[0] ### for practical use
print('DeltaZ2E,DeltaZ2I:',DeltaZ2E,DeltaZ2I)
DeltaZ2 = np.zeros((N,N))
DeltaZ2[:,:NE], DeltaZ2[:,NE:] = DeltaZ2E, DeltaZ2I

### E population and I population separately
Z2E_E, Z2I_E = N*J**2*hat_sigmae**2*tau_series*ALPHAE, -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2E_I, Z2I_I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE, N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E_E, DeltaZ2I_E = Z2E_E[1]-Z2E_E[0],Z2I_E[1]-Z2I_E[0] ### for practical use
DeltaZ2E_I, DeltaZ2I_I = Z2E_I[1]-Z2E_I[0],Z2I_I[1]-Z2I_I[0] ### for practical use
DeltaZ2_E, DeltaZ2_I = np.zeros((N,N)),np.zeros((N,N))
DeltaZ2_E[:,:NE], DeltaZ2_E[:,NE:] = DeltaZ2E_E, DeltaZ2I_E

eigvJ0: (-1.8073922282301493+0j)  theory: -1.8073922282301278
DeltaZ2E,DeltaZ2I: -0.00046666666666666655 0.0031733333333333336


### Chain motifs

In [ ]:
ntau   = 10#21#
trials = 30+6#6
tau_series = np.linspace(0,0.225,ntau)# np.linspace(0.25,0.4,ntau)#

In [12]:
alphachn_series  = np.array([0.000,0.100,0.200,0.300,0.400,0.500,0.600,0.700,0.800,0.900])
alpha_series_rec = np.array([0.000,0.200,0.400,0.600,0.800,1.000,1.200,1.400,1.600,1.800])

In [13]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@x+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)
shiftx = 1.5
def odeIntegralP(x,t,J,I=0):
	x = np.squeeze(x)
	x = np.reshape(x,(len(x),1))
	# print('size:',np.shape(x),np.shape(J@np.tanh(x)))
	dxdt = -x+J@(1.0+np.tanh(x-shiftx))
	return np.squeeze(dxdt)
def odesimulationP(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegralP,J=Jpt,I=I),xinit,t)

In [37]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

In [38]:
ce=ci=c
### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

[0.00556416 0.0056629 ]


In [39]:
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num   = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population


#### multiple values

In [41]:
trials = 36
alphachns     = ['0.000','0.100','0.200','0.300','0.400','0.500','0.600','0.700','0.800','0.900']
alphachns_rec = ['0.000','0.200','0.400','0.600','0.800','1.000','1.200','1.400','1.600','1.800']
# alphachns_arr = np.array([0.1,0.2,0.3,0.4])
eigvchn_series = np.zeros((trials,len(alphachns),N),dtype=complex)
eiglvec0norm_series= np.zeros((trials,ntau,N,2))
leig0mean_series = np.zeros((trials,ntau,N,2))
leig0pre_series = np.zeros((trials,ntau,N,2))
norm_4rvec_series, norm_4lvec_series = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
norm_4lvec_series_ = np.zeros((trials,ntau,2))

intg_mean_series  = np.zeros((trials,ntau,N,2,3))### mean overlap matrix rank and 
# tauchns_arr = alphachns_arr*c**2/c/(1-c)
seedss = np.array([1, 4, 8, 10, 13, 14, 18, 19, 25, 26, 31, 32, 34, 36, 40, 43, 44, 49, 52, 53, 55 ,60, 62, 64, 71, 73, 76, 78, 81, 82, 83,84, 85, 86, 87, 88, 93, 94, 95])
seedss = np.arange(1,37)
for ktrial in range(trials):
    seed =seedss[ktrial]
    z_pre = 0
    for idxalpha, alphachn in enumerate(alphachns[:]):
        # if idxalpha <8:
        #     continue
        if idxalpha == 0:       
            file_path = 'J:\\Allen_motif\\PRXL_data\\sonets\\N5000\\seed_'+str(seed)+'\\w_2000_0.200_'+alphachns_rec[idxalpha]+'_'+alphachn+'_'+alphachn+'_0.000_'+str(seed)+'.dat'
        else:
            file_path = 'J:\\Allen_motif\\PRXL_data\\sonets\\N5000\\seed_'+str(seed)+'\\w_2000_0.200_'+alphachns_rec[idxalpha]+'_'+alphachn+'_'+alphachn+'_1.000_'+str(seed)+'.dat'
        Jmat = np.zeros((N,N))
        count =0
        with open(file_path, 'r') as file:
            content = file.read()
            if content == '0':
                print('File is empty') 
            else:
                for line in content.split('\n'):
                    if line:
                        Jmat[count,:]=([(x) for x in line.split()])
                        count +=1
        # ### adding excitatory and inhibitory  
        # Zmat2 = (Jmat-c)@(Jmat-c)
        # chainsum = 0
        # for i in range(1000):
        #     for j in range(1000):
        #         if i!=j:
        #             chainsum += Zmat2[i,j]
        # print('chainsum,',chainsum/1000/999)           
        Jmat[:,:NE] = Jmat[:,:NE]*je
        Jmat[:,NE:] = Jmat[:,NE:]*(-ji)

        ktrial, it = ktrial, idxalpha
        ### full rank simulation
        xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
        xc_temporal = odesimulation(tt, xinit, Jmat, Inp)
        firing_rateeq[ktrial,it,:] = xc_temporal[-1,:].copy()
            
        zr = Jmat - Jbar
        ### compute the eigenvalues and eigenvectors of the matrix
        eigvchn, eigrvec = la.eig(Jmat)
        ## sort the eigenValues by it's norm of the real part 
        idxeigv = np.argsort(np.abs(eigvchn))
        idxeigv = idxeigv[::-1]
        eigvchn, eigrvec = eigvchn[idxeigv],eigrvec[:,idxeigv]
        
        eigvchn_,eiglvec = la.eig(Jmat.copy().T)
        idxeigv = np.argsort(np.abs(eigvchn_))
        idxeigv = idxeigv[::-1]
        eigvchn_, eiglvec = eigvchn[idxeigv],eiglvec[:,idxeigv]
        
        ### normalization
        reig    = np.squeeze(eigrvec[:,:].copy())
        leig0   = np.squeeze(eiglvec[:,:].copy()) 
        normval = np.sum(reig.copy()*leig0.copy(),axis=0)
        
        norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
        normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1
        if np.mean(reig[:NE,0])<0:
            reig[:,0]*=-1
            leig[:,0]*=-1
            leig0[:,0]*=-1
        if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
            reig[:,1]*=-1
            leig[:,1]*=-1
            leig0[:,1]*=-1
            
        ### numerical low-rank approximation 
        ov_inp_lowrank,ov_inp_lowrank_div = np.zeros(2),np.zeros(2)
        vec_lowrank_contribution = np.zeros((N,2))
        ### linear response theory approximation 
        for i in range(2):
            ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]   
            ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
        Equilibrium_lowrank_outliers = np.reshape(Inp.copy(),(N,1))
        # print('EQ shape:',np.shape(Equilibrium_lowrank_outliers))
        for i in range(2):
            vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
            # print('shape :...',np.shape(np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))))
            Equilibrium_lowrank_outliers += np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))
            #### REDUCE TO 2 POPULATION, THEREFORE RANK, POPULATION
            contributions_lr_num[ktrial,it,i,0] = np.mean(vec_lowrank_contribution[:NE,i])
            contributions_lr_num[ktrial,it,i,1] = np.mean(vec_lowrank_contribution[NE:,i])
        lowrank_eq_num[ktrial,it,:] = np.squeeze(Equilibrium_lowrank_outliers.copy())
        ovs_inplr_num[ktrial,it,:],ovs_inplr_div_num[ktrial,it,:] = ov_inp_lowrank.copy(),ov_inp_lowrank_div.copy() 
        
              
        print('seed:',seed,'alphachn:',alphachn,'idxalpha:',idxalpha)
        print('eigenvalues',eigvchn[:2])
        
        DELTA_Z = np.zeros((N,N))
        DELTA_Z = zr-z_pre  
        hzr = DELTA_Z.copy()
        
        eigvchn_series[ktrial,it,:] = eigvchn[:] 
        eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
        eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
        eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#
        for iii in range(2):
            eiglvec0norm_series[ktrial,it,:,iii] = leig0[:,iii].copy()/normval[iii,iii]
            
        DeltaZ2 = hzr@hzr    ### used to correct        
        if it<1:
            lvec, rvec = np.squeeze(leigvec0[:,:2]),np.squeeze(reigvec0[:,:2])
            # eigeng     = np.squeeze(eigvchn_series[ktrial,it,:2].copy())
            eigeng     = np.array([JE-JI,0])
            lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
        else:
            lvec, rvec = np.squeeze(eiglvec0_series[ktrial,it-1,:,:2]),np.squeeze(eigrvec_series[ktrial,it-1,:,:2]) ### use the previous eigenvector as the initial condition   
            eigeng = np.squeeze(eigvchn_series[ktrial,it-1,:2].copy())
            ### conditioned mean  
            lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
            lvec_mean[:NE,:],lvec_mean[NE:,:]=np.mean(lvec_mean[:NE,:],axis=0),np.mean(lvec_mean[NE:,:],axis=0)
            rvec_mean[:NE,:],rvec_mean[NE:,:]=np.mean(rvec_mean[:NE,:],axis=0),np.mean(rvec_mean[NE:,:],axis=0)
            
        ### get the appropriate normalization factor
        norm_for_lvec, norm_for_rvec = np.zeros(2),np.zeros(2)
        ### for the first delta tau, there is no change in the structure of the eigenvectors (only the random part changes due to the iid random connectivity)
        if it==0:
            hzr_u = zr.copy()
            with_chn = 0
        else:
            hzr_u = hzr.copy()
            with_chn = 1
                    
        ### predicted eigenvectors based on the previous eigenvectors(not the real one)
        ### record the reconstructed eigenvectors
        intg_ml, intg_mr = leigvec0.copy(), reigvec0.copy()
        norm_rvec_temp, norm_lvec_temp = np.zeros((N,2)), np.zeros((N,2))
        for i in range(2):
            rvec_n = np.reshape(rvec[:,i],(-1,1)) + (hzr_u)@np.reshape(rvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
            lvec_n = np.reshape(lvec[:,i],(-1,1)) + (hzr_u).T@np.reshape(lvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
    
            ''' Mean connectivity '''                
            current_eigv = eigvchn_series[ktrial,it,i].copy()
            if it==0:
                intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                ''' otherwise no correlation can be calculated '''
                norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
            elif it < 5 and i==1:#it < 3 and i==1:### 7 is 2 blocks after radius<outlier
                ### original values are obtained numerically
                intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                ''' otherwise no correlation can be calculated '''
                norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
            else:
                eigenvalue_u = np.real(current_eigv)
                eigenvalue_um = np.real(eigeng[i]) 
                
                ### norm_for_rvec and norm_for_lvec 
                rmean_tmp =np.reshape(rvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*(DeltaZ2@np.reshape(rvec_mean[:,i],(-1,1)))/np.real(eigenvalue_um)**2,(N,1))### mean-shifting 
                rvec_n[:NE,0] = rvec_n[:NE,0] - np.mean(rvec_n[:NE,0])+rmean_tmp[:NE,0]
                rvec_n[NE:,0] = rvec_n[NE:,0] - np.mean(rvec_n[NE:,0])+rmean_tmp[NE:,0]
                
                ### raw, unnormalized
                lmean_tmp = np.reshape(lvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*np.reshape(lvec_mean[:,i],(1,-1))@DeltaZ2/np.real(eigeng[i])**2,(N,1))### mean-shifting 
                lvec_n[:NE,0] = lvec_n[:NE,0] - np.mean(lvec_n[:NE,0])+lmean_tmp[:NE,0]
                lvec_n[NE:,0] = lvec_n[NE:,0] - np.mean(lvec_n[NE:,0])+lmean_tmp[NE:,0]
                
                norm_for_rvec[i] = la.norm(rvec_n) ### normalization factor 
                norm_rvec_n = np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i] 
                norm_rvec_temp[:,i]=np.squeeze(norm_rvec_n.copy())
                
                norm_for_lvec[i] = np.squeeze(np.reshape(lvec_n,(1,-1))@np.reshape(norm_rvec_temp[:,i],(-1,1)))     
                norm_lvec_n = np.reshape(lvec_n.copy(),(-1,1))/norm_for_lvec[i] 
                ### need to be re-normalized
                norm_lvec_temp[:,i]  = np.squeeze(norm_lvec_n.copy())
                norm_lvec_temp[:,i] *= eigvchn_series[ktrial,it,i].real
                
                ### more simplified version using lvec_mean and rvec_mean
                intg_ml[:,i] = lmean_tmp[:,0]/norm_for_lvec[i]*eigenvalue_u
                leig0mean_series[ktrial,it,:,i] =lmean_tmp[:,0]/norm_for_lvec[i]
                leig0pre_series[ktrial,it,:,i] =np.reshape(with_chn*np.reshape(lvec_mean[:,i],(1,-1))@DeltaZ2/np.real(eigeng[i])**2,(N,1))[:,0]#*eigenvalue_u#
                intg_mr[:,i] = rmean_tmp[:,0]/norm_for_rvec[i]
                
                tilden = np.reshape(lvec_n.copy(),(-1,1))/norml0_series[ktrial,it,i]
                norm_4lvec_series_[ktrial,it,i]=np.reshape(tilden,(1,-1))@np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i]
                
                norm_4rvec_series[ktrial,it,i],norm_4lvec_series[ktrial,it,i]=norm_for_rvec[i],norm_for_lvec[i]
                if i==0:
                    print(i,'should be the same',norm_4lvec_series[ktrial,it,i]/norml0_series[ktrial,it,i],norm_4lvec_series_[ktrial,it,i])
                
            
            ### @YS 17 Nov, no matter what the variable it is.
            ### modify the mean of the elements on the left and right eigenvectors
            norm_rvec_temp[:NE,i] -= np.mean(norm_rvec_temp[:NE,i])
            norm_rvec_temp[NE:,i] -= np.mean(norm_rvec_temp[NE:,i])
            norm_rvec_temp[:NE,i] += np.mean(intg_mr[:NE,i])
            norm_rvec_temp[NE:,i] += np.mean(intg_mr[NE:,i])
            norm_lvec_temp[:NE,i] -= np.mean(norm_lvec_temp[:NE,i])
            norm_lvec_temp[NE:,i] -= np.mean(norm_lvec_temp[NE:,i])
            norm_lvec_temp[:NE,i] += np.mean(intg_ml[:NE,i])
            norm_lvec_temp[NE:,i] += np.mean(intg_ml[NE:,i])
                
            ### record the reconstructed eigenvectors
            eigrvec_series_rec[ktrial,it,:,i] = norm_rvec_temp[:,i].copy()
            eiglvec_series_rec[ktrial,it,:,i] = norm_lvec_temp[:,i].copy()
            ### record the mean of the elements on the left and right eigenvectors  
            intg_mean_series[ktrial,it,:,i,0] = np.reshape(intg_ml[:,i],(N,))
            intg_mean_series[ktrial,it,:,i,1] = np.reshape(intg_mr[:,i],(N,))
            
        '''overlap per population '''
        ### theoretically compute the equilibrium population-averaged firing rate
        an = np.zeros((2,2),dtype=complex)
        am = np.zeros((2,2),dtype=complex) ## population X rank    
        for ir in range(2):
            an[0,ir] = np.mean(intg_ml[:NE,ir])
            an[1,ir] = np.mean(intg_ml[NE:,ir])
            am[0,ir] = np.mean(intg_mr[:NE,ir])
            am[1,ir] = np.mean(intg_mr[NE:,ir])
            
        ### overlap sum
        overlap_inp = np.zeros(2,dtype=complex) 
        for ir in range(2):
            overlap_inp[ir] = (NE*an[0,ir]*Inp[0]+NI*an[1,ir]*Inp[-1])
            ovs_inplr[ktrial,it,ir] = overlap_inp[ir] ### recording_theory
            overlap_inp[ir]/= (1.0-eigvchn[ir])
            ovs_inplr_div[ktrial,it,ir] = overlap_inp[ir] ### recording theory
        eq_fr = np.zeros(2)
        eq_fr[0],eq_fr[1] = Inp[0],Inp[-1]
        for ir in range(2): ### rank two 
            ### record 
            contributions_lr[ktrial,it,ir,0] = overlap_inp[ir]*am[0,ir] ### excitatory population
            contributions_lr[ktrial,it,ir,1] = overlap_inp[ir]*am[1,ir] ### inhibitory population
            eq_fr[0] += am[0,ir]*overlap_inp[ir]
            eq_fr[1] += am[1,ir]*overlap_inp[ir]
        lowrank_eq[ktrial,it,:] = eq_fr.copy() 
    
        
        z_pre = zr.copy()    

C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:72: ComplexWarning: Casting complex values to real discards the imaginary part
  norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:89: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:90: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:94: ComplexWarning: Casting complex values to real discards the imaginary part
  vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:112: ComplexWarning: Casting complex values to real d

seed: 1 alphachn: 0.000 idxalpha: 0
eigenvalues [-2.60716489+0.j -0.75362268+0.j]
seed: 1 alphachn: 0.100 idxalpha: 1
eigenvalues [-2.72780651+0.j          0.72945757+0.16251001j]
0 should be the same 1.1170412195674284 1.1170412195674293


C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:161: ComplexWarning: Casting complex values to real discards the imaginary part
  intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:162: ComplexWarning: Casting complex values to real discards the imaginary part
  intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:163: ComplexWarning: Casting complex values to real discards the imaginary part
  leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:165: ComplexWarning: Casting complex values to real discards the imaginary part
  norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:166: ComplexWarning

seed: 1 alphachn: 0.200 idxalpha: 2
eigenvalues [-2.86796031+0.j  0.76799121+0.j]
0 should be the same 1.0444291895962432 1.044429189596242
seed: 1 alphachn: 0.300 idxalpha: 3
eigenvalues [-3.15396211+0.j  0.77311612+0.j]
0 should be the same 1.0931312575221925 1.093131257522193
seed: 1 alphachn: 0.400 idxalpha: 4
eigenvalues [-3.33806291+0.j  0.74824333+0.j]
0 should be the same 1.038743012152838 1.0387430121528367
seed: 1 alphachn: 0.500 idxalpha: 5
eigenvalues [-3.58053098+0.j  0.73391828+0.j]
0 should be the same 1.0556515213827364 1.0556515213827373
seed: 1 alphachn: 0.600 idxalpha: 6
eigenvalues [-3.80188088+0.j  0.81781372+0.j]
0 should be the same 1.059609964025238 1.059609964025238
seed: 1 alphachn: 0.700 idxalpha: 7
eigenvalues [-3.85711251+0.j  0.90622677+0.j]
0 should be the same 0.9938420632876225 0.9938420632876223


C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:74: RuntimeWarning: divide by zero encountered in divide
  leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1
C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:74: RuntimeWarning: invalid value encountered in divide
  leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1


seed: 1 alphachn: 0.800 idxalpha: 8
eigenvalues [-4.03045014+0.j  0.99222897+0.j]
0 should be the same 1.0328526451756486 1.0328526451756488
seed: 1 alphachn: 0.900 idxalpha: 9
eigenvalues [-4.20507337+0.j  1.28219166+0.j]
0 should be the same 1.040875112823858 1.0408751128238591
seed: 2 alphachn: 0.000 idxalpha: 0
eigenvalues [-2.83486285+0.j  0.76563091+0.j]
seed: 2 alphachn: 0.100 idxalpha: 1
eigenvalues [-3.11820778+0.j  0.75302426+0.j]
0 should be the same 1.110675563717098 1.1106755637170975
seed: 2 alphachn: 0.200 idxalpha: 2
eigenvalues [-3.3811189 +0.j          0.33147468+0.67372435j]
0 should be the same 1.0780819411025189 1.0780819411025193
seed: 2 alphachn: 0.300 idxalpha: 3
eigenvalues [-3.63862801+0.j          0.75813424+0.03291219j]
0 should be the same 1.0488219486180856 1.0488219486180854
seed: 2 alphachn: 0.400 idxalpha: 4
eigenvalues [-3.85380638+0.j          0.74283425-0.12091833j]
0 should be the same 1.0381076846189325 1.038107684618932
seed: 2 alphachn: 0.500 idx

C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:148: RuntimeWarning: invalid value encountered in divide
  lvec_n = np.reshape(lvec[:,i],(-1,1)) + (hzr_u).T@np.reshape(lvec[:,i].copy(),(-1,1))/np.real(eigeng[i])


seed: 5 alphachn: 0.100 idxalpha: 1
eigenvalues [-3.14357559+0.j          0.3490991 -0.66077128j]
0 should be the same 1.1160126237229837 1.116012623722983
seed: 5 alphachn: 0.200 idxalpha: 2
eigenvalues [-3.36078201+0.j          0.76420565+0.05515066j]
0 should be the same 1.0476279386646037 1.0476279386646046
seed: 5 alphachn: 0.300 idxalpha: 3
eigenvalues [-3.48222465+0.j          0.70444989+0.34872398j]
0 should be the same 1.0391343406049487 1.0391343406049487
seed: 5 alphachn: 0.400 idxalpha: 4
eigenvalues [-3.70666324+0.j          0.74261209+0.16212085j]
0 should be the same 1.0598646604236344 1.0598646604236341
seed: 5 alphachn: 0.500 idxalpha: 5
eigenvalues [-3.95506636+0.j  0.95422212+0.j]
0 should be the same 1.0486408831648686 1.048640883164868
seed: 5 alphachn: 0.600 idxalpha: 6
eigenvalues [-4.19101717+0.j  1.12714826+0.j]
0 should be the same 1.0463507375084569 1.0463507375084566
seed: 5 alphachn: 0.700 idxalpha: 7
eigenvalues [-4.40102208+0.j  1.2480815 +0.j]
0 should b

C:\Users\Administrator\AppData\Local\Temp\ipykernel_138312\698358706.py:147: RuntimeWarning: invalid value encountered in divide
  rvec_n = np.reshape(rvec[:,i],(-1,1)) + (hzr_u)@np.reshape(rvec[:,i].copy(),(-1,1))/np.real(eigeng[i])


seed: 14 alphachn: 0.100 idxalpha: 1
eigenvalues [-3.34580736+0.j          0.68192638-0.32336344j]
0 should be the same 1.210919997542487 1.2109199975424871
seed: 14 alphachn: 0.200 idxalpha: 2
eigenvalues [-3.81915586+0.j          0.23655642+0.71952444j]
0 should be the same 1.103672208672743 1.103672208672743
seed: 14 alphachn: 0.300 idxalpha: 3
eigenvalues [-4.15554297+0.j          0.62319679+0.40824714j]
0 should be the same 1.0477319000167973 1.0477319000167977
seed: 14 alphachn: 0.400 idxalpha: 4
eigenvalues [-4.48039741+0.j         0.60638108+0.4206456j]
0 should be the same 1.0342154601130455 1.0342154601130447
seed: 14 alphachn: 0.500 idxalpha: 5
eigenvalues [-4.84388461+0.j          0.74474756-0.13464883j]
0 should be the same 1.0536047256176095 1.0536047256176093
seed: 14 alphachn: 0.600 idxalpha: 6
eigenvalues [-5.06932074+0.j  1.21362725+0.j]
0 should be the same 1.0219962690263857 1.021996269026386
seed: 14 alphachn: 0.700 idxalpha: 7
eigenvalues [-5.31683938+0.j  1.45664

#### single value

In [58]:
''' Parameters used for all networks (reciprocal and chain)'''
N = 5000#1750#2000
J = 1/np.sqrt(1500)*0.5
### get the current path location and read the data file
import os
strr= os.getcwd()
print(strr)

c = 0.1#0.2
# sigma2 = N*c*(1-c)*J**2
g, gamma = 6.8, 1/4.0#1.5,1/1.# 6.8, 1/4.0#1.5,1/1.#
NE = int(N/(1+gamma))
NI = int(gamma*NE)
N  = NE+NI
print(N,NE,NI)
ALPHAE, ALPHAI = NE/N, NI/N
KE, KI = int(c*NE), int(c*NI) ### fixed out-degree
print('number of connected E/I neurons:',KE,KI)
print('non-zero J:',J)
ji,je = g*J,J 

### define the network parameters of the diluted Gaussian matrix 
ge, gi = np.sqrt(je**2*c*(1-c)*N), np.sqrt(ji**2*c*(1-c)*N) 
hat_sigmae, hat_sigmai = np.sqrt(c*(1-c)), np.sqrt(c*(1-c))### standard deviation of the adjacency matrix
sigmae,sigmai = np.sqrt(c*(1-c)*J**2*N), np.sqrt(c*(1-c)*(-g*J)**2*N)### with magnitude of the coupling
JE,JI = je*c*NE, ji*c*NI 
lambda0 = JE-JI 
print('lambda0:',lambda0)
print('ge,gi:',ge,gi)
print('JE,JI:',JE,JI)

### construct the mean matrix


e:\Dropbox\DailyWork\Allen_project\sonets
5000 4000 1000
number of connected E/I neurons: 400 100
non-zero J: 0.012909944487358056
lambda0: -3.6147844564602556
ge,gi: 0.27386127875258304 1.862256695517565
JE,JI: 5.163977794943222 8.778762251403478


In [43]:
data = np.load("E:\Dropbox\DailyWork\Allen_project\preparation4paper_Data\sonets\ConnStats_WD_Adjacency_4April_PRX_68N1000_Nscaling_c03alpha06.npz",allow_pickle=True)
eigvchn_series_1 = data['eigvchn_series']
data = np.load("E:\Dropbox\DailyWork\Allen_project\preparation4paper_Data\sonets\ConnStats_WD_Adjacency_4April_PRX_68N2000_Nscaling_c015alpha06.npz",allow_pickle=True)
eigvchn_series_2 = data['eigvchn_series']
data = np.load("E:\Dropbox\DailyWork\Allen_project\preparation4paper_Data\sonets\ConnStats_WD_Adjacency_4April_PRX_68N3000_Nscaling_c01alpha06.npz",allow_pickle=True)
eigvchn_series_3 = data['eigvchn_series']
data = np.load("E:\Dropbox\DailyWork\Allen_project\preparation4paper_Data\sonets\ConnStats_WD_Adjacency_4April_PRX_68N4000_Nscaling_c0075alpha06.npz",allow_pickle=True)
eigvchn_series_4 = data['eigvchn_series']

In [45]:
np.shape(eigvchn_series_1)

(36, 1, 1000)

In [53]:
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1000_2000_FIXNUM.npz"

In [55]:
data = np.load(data_name,allow_pickle=True)
eigvchn_series_NscaledProb = data['eigvchn_series_NscaledNum']

In [49]:
np.shape(eigvchn_series_NscaledProb)

(5, 36, 2)

In [56]:
fig,ax = plt.subplots(figsize=(6,4))
series1=np.array([np.mean(eigvchn_series_1[:,0,0].real),np.mean(eigvchn_series_2[:,0,0].real),np.mean(eigvchn_series_3[:,0,0].real),np.mean(eigvchn_series_4[:,0,0].real),np.mean(eigvchn_series_4[:,0,0].real)])
series2=np.array([np.mean(eigvchn_series_1[:,0,1].real),np.mean(eigvchn_series_2[:,0,1].real),np.mean(eigvchn_series_3[:,0,1].real),np.mean(eigvchn_series_4[:,0,1].real),np.mean(eigvchn_series_4[:,0,1].real)])

ax.plot([1000,2000,3000,4000,5000],series1,'-',color='blue',linewidth=2.5,label='real')
ax.plot([1000,2000,3000,4000,5000],series2,'-',color='red',linewidth=2.5,label='imaginary')

In [57]:
ax.plot([1000,2000,3000,4000,5000],np.mean(eigvchn_series_NscaledProb[:,:,0].real,axis=1),'--',color='blue',linewidth=2.5,label='real')
ax.plot([1000,2000,3000,4000,5000],np.mean(eigvchn_series_NscaledProb[:,:,1].real,axis=1),'--',color='blue',linewidth=2.5,label='real')

In [60]:
trials = 36
alphachns     = ['0.000','0.100','0.200','0.300','0.400','0.500','0.600','0.700','0.800','0.900']
alphachns_rec = ['0.000','0.200','0.400','0.600','0.800','1.000','1.200','1.400','1.600','1.800']
# alphachns_arr = np.array([0.1,0.2,0.3,0.4])
eigvchn_series = np.zeros((trials,1,N),dtype=complex)
# tauchns_arr = alphachns_arr*c**2/c/(1-c)
seedss = np.array([1, 4, 8, 10, 13, 14, 18, 19, 25, 26, 31, 32, 34, 36, 40, 43, 44, 49, 52, 53, 55 ,60, 62, 64, 71, 73, 76, 78, 81, 82, 83,84, 85, 86, 87, 88, 93, 94, 95])
seedss = np.arange(1,37)
seedss = np.arange(0,36)
for ktrial in range(trials):
    seed =seedss[ktrial]
    z_pre = 0
    idxalpha,alphachn=6,0.6

    # file_path = 'K:Projects\\Allen_motif\\PRXL_data\\sonets\\N5000\\seed_'+str(seed)+'\\w_1750_0.171_'+alphachns_rec[idxalpha]+'_'+alphachns[idxalpha]+'_'+alphachns[idxalpha]+'_1.000_'+str(seed)+'.dat'
    file_path = 'K:\\Projects\\Allen_motif\\PRXL_data\\sonets\\N5000_FIXNUM\\seed_'+str(seed)+'\\w_5000_0.060_'+alphachns_rec[idxalpha]+'_'+alphachns[idxalpha]+'_'+alphachns[idxalpha]+'_1.000_'+str(seed)+'.dat'
    Jmat = np.zeros((N,N))
    count =0
    with open(file_path, 'r') as file:
        content = file.read()
        if content == '0':
            print('File is empty') 
        else:
            for line in content.split('\n'):
                if line:
                    Jmat[count,:]=([(x) for x in line.split()])
                    count +=1
    # ### adding excitatory and inhibitory  
    # Zmat2 = (Jmat-c)@(Jmat-c)
    # chainsum = 0
    # for i in range(1000):
    #     for j in range(1000):
    #         if i!=j:
    #             chainsum += Zmat2[i,j]
    # print('chainsum,',chainsum/1000/999)           
    Jmat[:,:NE] = Jmat[:,:NE]*je
    Jmat[:,NE:] = Jmat[:,NE:]*(-ji)

    ktrial, it = ktrial, 0

    ### compute the eigenvalues and eigenvectors of the matrix
    eigvchn, eigrvec = la.eig(Jmat)
    ## sort the eigenValues by it's norm of the real part 
    idxeigv = np.argsort(np.abs(eigvchn))
    idxeigv = idxeigv[::-1]
    eigvchn, eigrvec = eigvchn[idxeigv],eigrvec[:,idxeigv]
    
    eigvchn_,eiglvec = la.eig(Jmat.copy().T)
    idxeigv = np.argsort(np.abs(eigvchn_))
    idxeigv = idxeigv[::-1]
    eigvchn_, eiglvec = eigvchn[idxeigv],eiglvec[:,idxeigv]
    
    ### normalization
    reig    = np.squeeze(eigrvec[:,:].copy())
    leig0   = np.squeeze(eiglvec[:,:].copy()) 
    normval = np.sum(reig.copy()*leig0.copy(),axis=0)
    
    normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1
    if np.mean(reig[:NE,0])<0:
        reig[:,0]*=-1
        leig[:,0]*=-1
        leig0[:,0]*=-1
    if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
        reig[:,1]*=-1
        leig[:,1]*=-1
        leig0[:,1]*=-1    
            
    print('seed:',seed,'alphachn:',alphachn,'idxalpha:',idxalpha)
    print('eigenvalues',eigvchn[:2])
    
    
    eigvchn_series[ktrial,it,:] = eigvchn[:] 

seed: 0 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.58900578+0.j          0.6296092 +0.50396022j]
seed: 1 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.86255169+0.j          0.65570826+0.49325813j]
seed: 2 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.33825762+0.j          0.79575151+0.08007646j]
seed: 3 alphachn: 0.6 idxalpha: 6
eigenvalues [-2.58725123+0.j          0.42140651+0.65334789j]
seed: 4 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.2630397 +0.j          0.59482493+0.50554347j]
seed: 5 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.75009058+0.j          0.53565355+0.61302222j]
seed: 6 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.00573454+0.j         -0.63327211+0.50738692j]
seed: 7 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.35507555+0.j  0.81438497+0.j]
seed: 8 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.05004016+0.j          0.81748283+0.03272334j]
seed: 9 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.41880615+0.j          0.35817431+0.73556524j]
seed: 10 alphachn: 0.6 idxalpha: 6
eigenvalues [-3.37696

In [65]:
# eigvchn_series_NscaledNum = np.zeros((5,trials,2),dtype=complex)


In [75]:
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1000_Nscaling_c03alpha06.npz"

data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']

In [76]:
eigvchn_series_NscaledNum[0,:,:] = eigvchn_series[:,0,:2].copy()

In [ ]:
eigvchn_series_NscaledProb = np.zeros((5,trials,2),dtype=complex)


In [100]:
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N5000_Nscaling_c02alpha06.npz"#ConnStats_WD_Adjacency_4April_PRX_68N2000_normlr_whilecode_used_vecrec.npz"#
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']

In [101]:
np.shape(eigvchn_series)

(36, 1, 5000)

In [102]:
eigvchn_series_NscaledProb[4,:,:] = eigvchn_series[:,0,:2].copy()

In [61]:
params = {'g':g,
          'gamma':gamma,
          'NE':NE,
          'NI':NI,
          'c':c,
          'J':J,
          }
lst = [eigvchn_series, params]
stg = ["eigvchn_series, params"]
data = list_to_dict(lst=lst, string=stg)
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N5000_Nscaling_c0060alpha06.npz"
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1000_Nscaling_c03alpha06.npz"
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz"
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N2000_normlr_whilecode_used_vecrec.npz"
np.savez(data_name, **data)
# data = np.load(data_name,allow_pickle=True)

# ### also loading other variables 
# eigvchn_series = data['eigvchn_series']

In [158]:
np.shape(eigvchn_series)

(36, 1, 2000)

In [77]:
eigvchn_theo

array([[-3.0840353 ,  0.91516462],
       [-3.0840353 ,  0.91516462],
       [-3.0840353 ,  0.91516462],
       [-3.0840353 ,  0.91516462],
       [-3.0840353 ,  0.91516462]])

In [103]:
eigvchn_theosss

array([[ -2.05602353,   0.61010975],
       [ -4.11204706,   1.2202195 ],
       [ -6.16807059,   1.83032925],
       [ -8.22409413,   2.44043899],
       [-10.28011766,   3.05054874]])

In [106]:
np.mean(eigvchn_series_NscaledProb,axis=1)

array([[ -2.83190387-0.01177341j,   0.92821352-0.00706514j],
       [ -4.24704828+0.j        ,   0.95958686+0.07097177j],
       [ -6.45546841+0.j        ,   1.43567191+0.02254239j],
       [ -8.54551146+0.j        ,   1.91276356+0.j        ],
       [-10.9659433 +0.j        ,   2.36808941+0.j        ]])

In [78]:
# eigvchn_series_NscaledProb[4,:,:] = eigvchn_series[:,6,:2]
lst = [eigvchn_series_NscaledNum, eigvchn_theo]
stg = ["eigvchn_series_NscaledNum, eigvchn_theo"]
data = list_to_dict(lst=lst, string=stg)
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1000_5000_FIXNUM.npz"
np.savez(data_name, **data)

In [107]:
# eigvchn_series_NscaledProb[4,:,:] = eigvchn_series[:,6,:2]
lst = [eigvchn_series_NscaledProb, eigvchn_theosss]
stg = ["eigvchn_series_NscaledProb, eigvchn_theosss"]
data = list_to_dict(lst=lst, string=stg)
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1000_5000_FIXPROB.npz"
np.savez(data_name, **data)

In [134]:
eigvchn_theosss

array([[-2.05602353,  0.61010975],
       [-2.57002941,  0.76263719],
       [-3.0840353 ,  0.91516462],
       [-3.59804118,  1.06769206],
       [-4.11204706,  1.2202195 ]])

In [24]:
NNs = np.array([3000,4000,5000])
NEs = NNs/5*4 
cs= 240/NEs
print(cs)

[0.1   0.075 0.06 ]


In [110]:
### plot the numerical error bar and theoretical eigvchn_theosss  
NNN = np.array([1000,1250,1500,1750,2000])  
NNN = np.array([1000,2000,3000,4000,5000])  
fig,ax=plt.subplots(figsize=(4,3))
alphass=0.95
ax.errorbar(NNN,np.nanmean(eigvchn_series_NscaledProb[:,:,0].real,axis=1),yerr=np.nanstd(eigvchn_series_NscaledProb[:,:,0].real,axis=1),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(NNN, eigvchn_theosss[:,0],marker='o',color='tab:red',alpha=1,lw=1.5)
ax.plot(NNN, eigvchn_theosss[:,0],color='tab:red',alpha=1,lw=1.5)

ax.errorbar(NNN,np.nanmean(eigvchn_series_NscaledProb[:,:,1].real,axis=1),yerr=np.nanstd(eigvchn_series_NscaledProb[:,:,1].real,axis=1),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(NNN, eigvchn_theosss[:,1],marker='o',color='tab:green',alpha=1,lw=1.5)
ax.plot(NNN, eigvchn_theosss[:,1],color='tab:green',alpha=1,lw=1.5)
### plot the y=1 line
ax.plot([900,5100],[1,1],'--',color='gray')

### design the axes
ax.set_xlim(900,5100)
ax.set_xticks(NNN)
ax.set_ylim(-11.5,5.5)
ax.set_yticks([-10,5])
### move the axes to the center
# ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{E}$',fontsize=14)
ax.tick_params(labelsize=12)
# ax.set_title(r'$\nu_{E}$',fontsi

In [162]:
lst = [eigvchn_series_NscaledNum, eigvchn_theo]
stg = ["eigvchn_series_NscaledNum, eigvchn_theo"]
data = list_to_dict(lst=lst, string=stg)
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1000_2000_FIXNUM.npz"
# np.savez(data_name, **data)

In [81]:
### plot the numerical error bar and theoretical eigvchn_theosss  
NNN = np.array([1000,2000,3000,4000,5000])  
fig,ax=plt.subplots(figsize=(4,3))
alphass=0.95
ax.errorbar(NNN,np.nanmean(eigvchn_series_NscaledNum[:,:,0].real,axis=1),yerr=np.nanstd(eigvchn_series_NscaledNum[:,:,0].real,axis=1),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(NNN, eigvchn_theo[:,0],marker='o',color='tab:red',alpha=1,lw=1.5)
ax.plot(NNN, eigvchn_theo[:,0],color='tab:red',alpha=1,lw=1.5)

ax.errorbar(NNN,np.nanmean(eigvchn_series_NscaledNum[:,:,1].real,axis=1),yerr=np.nanstd(eigvchn_series_NscaledNum[:,:,1].real,axis=1),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(NNN, eigvchn_theo[:,1],marker='o',color='tab:green',alpha=1,lw=1.5)
ax.plot(NNN, eigvchn_theo[:,1],color='tab:green',alpha=1,lw=1.5)

### plot the y=1 line
ax.plot([900,5100],[1,1],'--',color='gray')

### design the axes
ax.set_xlim(900,5100)
ax.set_xticks(NNN)
ax.set_ylim(-11.5,5.5)
ax.set_yticks([-10,5])
### move the axes to the center
# ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{E}$',fontsize=14)
ax.tick_params(labelsize=12)
# ax.set_title(r'$\nu_{E}$',fontsi

In [ ]:
for seed in {0..36}; do
./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed

done

In [ ]:

# for seed in {0..36}; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed

# done


# for seed in {1..37}; do
# ./run_secorder 2000 0.15 1.2 0.6 0.6 1 $seed
# done


# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed
# done

# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do     mkdir -p "seed_$seed"; done

In [15]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [86]:
# params = {'g':g,
#           'gamma':gamma,
#           'NE':NE,
#           'NI':NI,
#           'c':c,
#           'J':J,
#           'tau_series':tau_series,
#           }
# lst = [eigvchn_series, eigrvec_series, eiglvec_series,
#        eigrvec_series_rec, eiglvec_series_rec,
#         eiglvec0_series, norml0_series, params,
#         lowrank_eq, firing_rateeq,lowrank_eq_num,
#         ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,
#         contributions_lr, contributions_lr_num,
#         intg_mean_series,leig0mean_series,
#         norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_]
# stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
#        "eigrvec_series_rec, eiglvec_series_rec,"
#         "eiglvec0_series, norml0_series, params,"
#         "lowrank_eq, firing_rateeq,lowrank_eq_num,"
#         "ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,"
#         "contributions_lr, contributions_lr_num,"
#         "intg_mean_series,leig0mean_series,"
#         "norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_"]
# data = list_to_dict(lst=lst, string=stg)
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/19Mar_eigenvectors_corrected/Sparse_data/DynsStats_Adjacency_20Mar_16PRX_54N1000_normlr_whilecode.npz"
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz"
# np.savez(data_name, **data)
### load the stored data, in particular, extracting variable: eigvchn_series 
data = np.load(data_name,allow_pickle=True)

### also loading other variables 
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eigrvec_series_rec = data['eigrvec_series_rec']
eiglvec_series_rec = data['eiglvec_series_rec']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_mean_series = data['intg_mean_series']
leig0mean_series = data['leig0mean_series']
# ### load data
# data = np.load(data_name,allow_pickle=True)
# firing_rateeq = data['firing_rateeq']
# lowrank_eq    = data['lowrank_eq']
# lowrank_eq_num = data['lowrank_eq_num'] 
# contributions_lr = data['contributions_lr'] ### trial, tau, rank, pop
# contributions_lr_num = data['contributions_lr_num']
# eigvchn_series = data['eigvchn_series']
# eigrvec_series = data['eigrvec_series']
# eiglvec_series = data['eiglvec_series']
# eiglvec0_series = data['eiglvec0_series']
# norml0_series = data['norml0_series']
# params = data['params']
# intg_ov_series = data['intg_ov_series']
# first_perturb_ov = data['first_perturb_ov']
# first_perturb_ovP = data['first_perturb_ovP']
# intg_mean_series = data['intg_mean_series']
# intg_std_series = data['intg_std_series']
# leig0mean_series = data['leig0mean_series']


In [160]:
# chainsum = 0
# for i in range(NE):
#     for j in range(NE):
#         if i!=j:
#             chainsum += Zmat2[i,j]

In [87]:
ce, ci = c,c
radius_theo_map = np.zeros(ntau)
for it, tau in enumerate(tau_series):
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
### theoretical prediction of the eigenvalues

org_k = J*N*(ALPHAE-g*ALPHAI)/2.0
eigvchn_theo = np.zeros((ntau,2))
eigvchn_theo[:,0] = org_k*(c+np.sqrt(c**2+4*c*(1-c)*tau_series))
eigvchn_theo[:,1] = org_k*(c-np.sqrt(c**2+4*c*(1-c)*tau_series))

In [88]:
eigvchn_theo,radius_theo_map

(array([[-2.16887067, -0.        ],
        [-2.36755653,  0.19868586],
        [-2.53935802,  0.37048734],
        [-2.69291288,  0.5240422 ],
        [-2.83303478,  0.6641641 ],
        [-2.96273244,  0.79386176],
        [-3.0840353 ,  0.91516462],
        [-3.19838898,  1.0295183 ],
        [-3.30686644,  1.13799576],
        [-3.41029002,  1.24141934]]),
 array([0.63397161, 0.61791909, 0.60143828, 0.58449294, 0.56704144,
        0.54903552, 0.5304187 , 0.51112425, 0.4910723 , 0.47016593]))

In [89]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

In [90]:
#### print trial by trial the first two eigenvallues
for ktrial in range(trials):
    eigvchn = eigvchn_series[ktrial,:,:].copy()
    eigvchn_real = (eigvchn)
# ### delete the 17th trial
# eigvchn_series = np.delete(eigvchn_series,17,axis=0) #for ConnStats_Adjacency_12OctDense_.npz
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [ 8 20 21 29 24  9 17 15  4  0 30 35 13 14  3 31  5 25 18 28 10 33 12 34
 26  1 16 23  7 22] 30


In [83]:
### plot the numerical and theoretical eigenvalues outliers
# fig,ax = plt.subplots(figsize=(6,4))
# ### label is r_{bulk}^{num}
# ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
# ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# # ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
# ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
# ax.set_xlabel(r'$\tau$')
# ax.set_ylabel('Radius')
# ax.legend()
# plt.show()

alphass= 0.25

ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',label='theoretical',alpha=alphass)
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',alpha=alphass)

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-6.5,3.5])
ax.set_yticks([-5,0,3])
# ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [84]:
### plot y==1 
ax.axhline(y=1, color='k', linestyle='-')

In [92]:
### plot a shifting histogram for the first eigenvalue
fig,ax=plt.subplots(figsize=(6,3))
### plot the mean of the eigenvalues, how it changes with tau_series
### colormap gradiently changes with the tau_series
cm = plt.cm.YlOrRd
### the mean of the first eigenvalue
mean_eigvchn = np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)
sc=ax.scatter(mean_eigvchn,-0.1*np.ones(ntau),c=tau_series,cmap=cm)
### the mean of the second eigenvalue 
cm = plt.cm.YlGn
mean_eigvchn = np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)
sc=ax.scatter(mean_eigvchn,0.1*np.ones(ntau),c=tau_series,cmap=cm)

ax.set_xlabel(r'$\Re(\lambda)$',fontsize=14)
### set double ylabels
ax.set_ylabel(r'$\Im(\lambda)$',fontsize=14)
ax2 = ax.twinx() 
 
color = 'tab:gray'
ax2.tick_params(axis ='y', labelcolor = color) 
ax2.set_yticks([])
ax2.spines['bottom'].set_color('none')
### set other x-axis invisible
ax2.spines['top'].set_color('none')
ax2.spines['right'].set_color('none')
ax2.spines['left'].set_color('none')
# ax.set_title('Eigenvalue spectrum',fontsize=14,fontweight='bold')
### refine the plot 
# ax.set_xlim([-3,2])
# ax.set_ylim([-0.3,0.3])
ax.set_xlim([-4,2.])
ax.set_ylim([-0.3,0.3])

ax.set_aspect('equal')
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
## set the x and y ticks
ax.set_xticks([-3,0,2])
ax.set_yticks([-0.5,0,0.5])
# ### also, plot the theoretical predictions for the first and second eigenvalues
# ax.scatter(eigvchn_theo[:,0].real,np.zeros(ntau),cmap=plt.cm.YlOrRd, c=tau_series, s=50,alpha=1,marker='^')
# ax.scatter(eigvchn_theo[:,1].real,np.zeros(ntau),cmap=plt.cm.YlGn, c=tau_series, s=50,alpha=1,marker='^')

### plot a circle with the radius of the last radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[-1],color='tab:blue',fill=False,linewidth=1.5,alpha=1))
### plot a circle with the radius of the first radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[0],color='tab:purple',fill=False,linewidth=1.5,alpha=0.5,linestyle='--'))

ktrial_index = 16
### scatter plot the eigenvalue bulk of this trial
### randomly select 500 points
idx_random = np.random.choice(np.arange(2,N),size=500,replace=False)
ax.scatter(eigvchn_series[ktrial_index,0,idx_random].real,eigvchn_series[ktrial_index,0,idx_random].imag,color='tab:purple', s=5,alpha=0.15)
ax.scatter(eigvchn_series[ktrial_index,-1,idx_random].real,eigvchn_series[ktrial_index,-1,idx_random].imag,color='tab:blue', s=5,alpha=0.2)
### also plot the original eigenvalue outlier: JE-JI, using red color and X marker
ax.scatter(JE-JI,0,color='black',marker='X',s=80,alpha=1)


In [ ]:
# ./run_secorder 1000 0.2 0.0 0.0 0.0 0 5
# ./run_secorder 1000 0.2 0.2 0.1 0.1 1 5
# ./run_secorder 1000 0.2 0.4 0.2 0.2 1 5
# ./run_secorder 1000 0.2 0.6 0.3 0.3 1 5
# ./run_secorder 1000 0.2 0.8 0.4 0.4 1 5
# ./run_secorder 1000 0.2 1.0 0.5 0.5 1 5
# ./run_secorder 1000 0.2 1.2 0.6 0.6 1 5
# ./run_secorder 1000 0.2 1.4 0.7 0.7 1 5
# ./run_secorder 1000 0.2 1.6 0.8 0.8 1 5
# ./run_secorder 1000 0.2 1.8 0.9 0.9 1 5

In [ ]:
# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed
# done

In [ ]:
for seed in {1..37}; do
./run_secorder 1500 0.15 1.4 0.7 0.7 1 $seed
done

In [95]:
tau_series,alpha_series

(array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
        0.225]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))

In [96]:
kE, kI = 240,60
css = np.zeros(2)
css[0]=240/800 
css[1]=240/1600
css

array([0.3 , 0.15])

In [100]:
alpha_sample=0.6
CHNNUM = (alpha_sample+1)*c**2*N**2
CHNNUM_ = (0.7+1)*0.2**2*1500**2
print(CHNNUM,CHNNUM_)
alphass = np.zeros(2)
alphass[0] = CHNNUM/1000**2/css[0]**2-1
alphass[1] = CHNNUM/2000**2/css[1]**2-1
print(alphass)

144000.00000000003 153000.0
[0.6 0.6]


In [62]:
# # ccccs = 300/np.array([1000,1250,1500,1750,2000])###（CE+CI)/N
# ccccs = 300/np.array([1000,2000,3000,4000,5000])###（CE+CI)/N(larger)
# print(ccccs)
# ### compute rhos 
# taussss = (ccccs**2*(1+0.6)-ccccs**2)/ccccs/(1-ccccs)
# print(taussss)
# Nssss = np.array([1000,1250,1500,1750,2000])
# Nssss = np.array([1000,2000,3000,4000,5000])
# print(Nssss)

ccccs = 300/np.array([1000,2000,3000,4000,5000])###（CE+CI)/N(larger)
print(ccccs)
### compute rhos 
taussss = (ccccs**2*(1+0.6)-ccccs**2)/ccccs/(1-ccccs)
print(taussss)
Nssss = np.array([1000,2000,3000,4000,5000])
print(Nssss)

[0.3   0.15  0.1   0.075 0.06 ]
[0.25714286 0.10588235 0.06666667 0.04864865 0.03829787]
[1000 2000 3000 4000 5000]


In [63]:
''' Parameters used for all networks (reciprocal and chain)'''

# c [0.3        0.24       0.2        0.17142857 0.15      ]
# tau [0.25714286 0.18947368 0.15       0.12413793 0.10588235]
# N [1000 1250 1500 1750 2000]
J = 1/np.sqrt(1500)*0.5
### get the current path location and read the data file
import os
strr= os.getcwd()
print(strr)

c = ccccs.copy()
# sigma2 = N*c*(1-c)*J**2
g, gamma = 6.8, 1/4.0#1.5,1/1.# 6.8, 1/4.0#1.5,1/1.#
ALPHAE, ALPHAI = 4/5, 1/5
ce, ci = c,c            
### theoretical prediction of the eigenvalues

org_k = J*Nssss*(ALPHAE-g*ALPHAI)/2.0
eigvchn_theo = np.zeros((len(ccccs),2))
eigvchn_theo[:,0] = org_k*(c+np.sqrt(c**2+4*c*(1-c)*taussss))
eigvchn_theo[:,1] = org_k*(c-np.sqrt(c**2+4*c*(1-c)*taussss))


e:\Dropbox\DailyWork\Allen_project\sonets


In [64]:
eigvchn_theo[:,0],eigvchn_theo[:,1]

(array([-3.0840353, -3.0840353, -3.0840353, -3.0840353, -3.0840353]),
 array([0.91516462, 0.91516462, 0.91516462, 0.91516462, 0.91516462]))

In [83]:
Nssss

array([1000, 2000, 3000, 4000, 5000])

In [82]:
''' Parameters used for all networks (reciprocal and chain)'''

# c [0.2      ........      ]
# tau [0.15 ......]
# N [1000 1250 1500 1750 2000]
J = 1/np.sqrt(1500)*0.5
### get the current path location and read the data file
import os
strr= os.getcwd()
print(strr)

c = np.ones(5)*0.2
taus = np.array([0.15,0.15,0.15,0.15,0.15])
# sigma2 = N*c*(1-c)*J**2
g, gamma = 6.8, 1/4.0#1.5,1/1.# 6.8, 1/4.0#1.5,1/1.#
ALPHAE, ALPHAI = 4/5, 1/5
ce, ci = c,c            
### theoretical prediction of the eigenvalues

org_k = J*Nssss*(ALPHAE-g*ALPHAI)/2.0
eigvchn_theosss = np.zeros((len(ccccs),2))
eigvchn_theosss[:,0] = org_k*(c+np.sqrt(c**2+4*c*(1-c)*taus))
eigvchn_theosss[:,1] = org_k*(c-np.sqrt(c**2+4*c*(1-c)*taus))

print(eigvchn_theosss[:,0])
print(eigvchn_theosss[:,1])

''' Parameters used for all networks (reciprocal and chain)'''

e:\Dropbox\DailyWork\Allen_project\sonets
[ -2.05602353  -4.11204706  -6.16807059  -8.22409413 -10.28011766]
[0.61010975 1.2202195  1.83032925 2.44043899 3.05054874]


' Parameters used for all networks (reciprocal and chain)'

In [48]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(4,2))
### label is r_{bulk}^{num}
ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

### I mean... eigvchn_series[:,:,0].real and eigvchn_series[:,:,1].real compared with eigvchn_theo[:,0] and eigvchn_theo[:,1]
# fig,ax = plt.subplots(figsize=(6,6))
ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',marker='o',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',marker='o')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,0].real,axis=0),'k',label='numerical')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,1].real,axis=0),'k')
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)-np.std(eigvchn_series[idx_eff,:,0].real,axis=0),np.mean(eigvchn_series[:,:,0].real,axis=0)+np.std(eigvchn_series[idx_eff,:,0].real,axis=0),color='tab:red',alpha=0.15)
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)-np.std(eigvchn_series[idx_eff,:,1].real,axis=0),np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)+np.std(eigvchn_series[idx_eff,:,1].real,axis=0),color='tab:green',alpha=0.15)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-4.0,2.5])
ax.set_yticks([-3,0,2])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [49]:
mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))

thl = 1.5
ths = 1.5

for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)

            mean_leig0vec_series[ktrial,it,ir,0] = np.mean(leig0mean_series[ktrial,it,:NE,ir])
            mean_leig0vec_series[ktrial,it,ir,1] = np.mean(leig0mean_series[ktrial,it,NE:,ir])
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6#3
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,0].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,1].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan

In [50]:
### rescaled sqrt(N)
mean_leigvec_num_series *=np.sqrt(N)
mean_reigvec_num_series *=np.sqrt(N)
mean_leigvec_series *=np.sqrt(N)
mean_reigvec_series *=np.sqrt(N)

In [51]:
dtau = tau_series[1]-tau_series[0]
### plot the mean of the left and right eigenvectors
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True,sharey=True)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.3)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.08)
# ax[0].set_yticks([0,0.05])
ax[0].set_ylim(-0.,1.5)
ax[0].set_yticks([0,1.5])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.45])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_ylim(-0.0,1.5)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)

fig.tight_layout()


In [52]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,0],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$E$')
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[idx_eff,:,0,0]*eigvchn_series[idx_eff,:,0],axis=0),color='black',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,1,0],axis=0),color='gray',s=10,label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),color='gray',s=10,marker='^',label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### excitatory population
### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[0].set_ylim(-0.2,0.35)
ax[0].set_yticks([0,0.3])

ax[0].set_ylim(-5,8)
ax[0].set_yticks([-5,8])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.2,0.4])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[1].set_ylim(-1.5,1)
# ax[1].set_yticks([-1,0])
ax[1].set_ylim(-40,15)
ax[1].set_yticks([-40,15])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [53]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N),facecolor='green',alpha=0.3)
## excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.18)
# ax[0].set_yticks([0,0.15])# noparadoxical
ax[0].set_ylim(-0.01,0.06)
ax[0].set_yticks([0,0.05])
# ax[0].set_ylim(-0.1,0.3)
# ax[0].set_yticks([-0.1,0.3])
ax[0].set_ylim(-0.3,0.1)
ax[0].set_yticks([-0.3,0.1])
# ax[0].set_ylim(-0.1,0.3)
# ax[0].set_yticks([-0.1,0.3])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population

# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,1],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N),facecolor='green',alpha=0.3)
## move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_ylim(-0.4,0.02)
ax[1].set_yticks([-0.4,0.])
# ax[1].set_ylim(-1,0.5)
# ax[1].set_yticks([-1,0.5])
ax[1].set_ylim(-0.5,1)
ax[1].set_yticks([-0.5,1])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [54]:
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
norm_4lvec_series_select = norm_4lvec_series.copy()
norm_4lvec_series_select_ = norm_4lvec_series_.copy()
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(norm_4lvec_series[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select[idxnan,it,ir] = np.nan 
        
        idxsort = np.argsort(norm_4lvec_series_[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select_[idxnan,it,ir] = np.nan 

In [55]:
#### plot the norm_4lvec and norm_4rvec change with tau^r   
fig, ax = plt.subplots(figsize=(4,2))
### plot error bar  
alphass = 0.95
ax.errorbar(tau_series[1:]-0.005,np.nanmean(norm_4lvec_series_select[:,1:,0]/norml0_series[:,1:-1,0],axis=0),yerr=np.nanstd(norm_4lvec_series_select[:,1:,0]/norml0_series[:,1:-1,0],axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
ax.errorbar(tau_series[1:]+0.005,np.mean(norm_4rvec_series[:,1:,0],axis=0),yerr=np.std(norm_4rvec_series[:,1:,0],axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
ax.set_xlim([tau_series[0]-0.05,tau_series[-1]+0.05])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([0.9,1.3])
ax.set_yticks([0.9,1.3])

#### plot the norm_4lvec and norm_4rvec change with tau^r   
fig, ax = plt.subplots(figsize=(4,2))
### plot error bar  
alphass = 0.95
ax.errorbar(tau_series[1:],np.nanmean(norm_4lvec_series_select_[:,1:,0].real,axis=0),yerr=np.nanstd(norm_4lvec_series_select_[:,1:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
ax.errorbar(tau_series[1:],np.mean(norm_4rvec_series[:,1:,0].real,axis=0),yerr=np.std(norm_4rvec_series[:,1:,0].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
ax.set_xlim([tau_series[0]-0.05,tau_series[-1]+0.05])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([0.9,1.3])
ax.set_yticks([0.9,1.3])

In [62]:
### plot random components on left and right eigenvectors when tau == 0.02 and ktrial = 15
ktrial = idx_eff[12]
its = [int((ntau-1)*2/3),ntau-1]
fig,ax=plt.subplots(1,2,figsize=(10,4),sharex=True,sharey=True)
for i,it in enumerate(its):
    ax[i].scatter(eigrvec_series[ktrial,it,:NE,0]*np.sqrt(N),eigrvec_series[ktrial,it,:NE,1]*np.sqrt(N),color='tab:red',s=10,label='right eigenvector '+r'$E$',alpha=0.25)
    ax[i].scatter(eigrvec_series[ktrial,it,NE:,0]*np.sqrt(N),eigrvec_series[ktrial,it,NE:,1]*np.sqrt(N),color='tab:blue',s=10,label='right eigenvector '+r'$I$',alpha=0.25)
    ### equal axis
    ax[i].set_aspect('equal')
    ax[i].set_xlim(-0.15,0.15)
    ax[i].set_ylim(-0.15,0.15)
    ax[i].set_xlim(-4.5,4.5)
    ax[i].set_ylim(-4.5,4.5)
    ax[i].set_xlabel('Random component rank 1',fontsize=14)
    ax[i].set_ylabel('Random component rank 2',fontsize=14)
    ax[i].legend()
    ax[i].set_title('Right eigenvector '+r'$\tau=$'+str(tau_series[it]),fontsize=14)
    ax[i].set_xticks([-0.1,0,0.1])
    ax[i].set_yticks([-0.1,0,0.1])
    ax[i].set_xticks([-2,0,2])
    ax[i].set_yticks([-2,0,2])
    ### remove the top and right axes
    ax[i].spines['top'].set_color('none')
    ax[i].spines['right'].set_color('none')
    ### move the x-axis and y-axis to the center
    ax[i].spines['bottom'].set_position(('data',0))
    ax[i].spines['left'].set_position(('data',0))
fig.tight_layout()


In [57]:
# ktrial = 8
### compute the slope of the right eigenvectors (correlated, rank-1 and rank-2)
### using linear regression and the slope is the first element of the output
from scipy.stats import linregress
slope_eigrvec = np.zeros((len(idx_eff),ntau,2))
slope_eigrvec_rec = np.zeros((len(idx_eff),ntau,2))
for ik,ktrial in enumerate(idx_eff):
    for it in range(ntau):
        slope_eigrvec[ik,it,0] = linregress(eigrvec_series[ktrial,it,:NE,0],eigrvec_series[ktrial,it,:NE,1])[0]
        slope_eigrvec_rec[ik,it,0] = linregress(eigrvec_series_rec[ktrial,it,:NE,0],eigrvec_series_rec[ktrial,it,:NE,1])[0]
        slope_eigrvec[ik,it,1] = linregress(eigrvec_series[ktrial,it,NE:,0],eigrvec_series[ktrial,it,NE:,1])[0]
        slope_eigrvec_rec[ik,it,1] = linregress(eigrvec_series_rec[ktrial,it,NE:,0],eigrvec_series_rec[ktrial,it,NE:,1])[0]
        
        if np.abs(slope_eigrvec[ik,it,0])>2:
            slope_eigrvec[ik,it,0] = np.nan
        if np.abs(slope_eigrvec[ik,it,1])>2:
            slope_eigrvec[ik,it,1] = np.nan
        if np.abs(slope_eigrvec_rec[ik,it,0])>2:
            slope_eigrvec_rec[ik,it,0] = np.nan
        if np.abs(slope_eigrvec_rec[ik,it,1])>2:
            slope_eigrvec_rec[ik,it,1] = np.nan
            
### plot the (trial-mean) slope of the right eigenvectors, change with tau
fig, ax = plt.subplots(figsize=(4,2))
ax.plot(tau_series,np.nanmean(slope_eigrvec[:,:,0],axis=0),color='black',linewidth=1.5,linestyle='--',label='numerical',marker='o')
ax.plot(tau_series,np.nanmean(slope_eigrvec_rec[:,:,0],axis=0),color='black',linewidth=1.5,label='reconstructed',marker='o')

ax.set_xlim(tau_series[0],tau_series[its[1]])
ax.set_ylim(-1.5,0.1)
ax.set_yticks([-1,0])
ax.set_xticks([tau_series[0],tau_series[its[1]]])
### move the x-axis to  the cecnter
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',tau_series[0]))
ax.set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax.set_ylabel('Slope of \nright eigenvector',fontsize=12)
ax.legend()
# fig.tight_layout()

In [60]:
#### same as above but for left eigenvectors
xlims = [-1.5,1.5]
ylims = [-1.5,1.5]
xlims = [-160,60]
ylims = [-160,60]
for iii in range(1,3,1):
    ktrial = idx_eff[iii]
    fig,ax=plt.subplots(1,2,figsize=(10,4),sharex=True,sharey=True)
    for i,it in enumerate(its):
        ax[i].scatter(eiglvec_series[ktrial,it,:NE,0]*eigvchn_series[ktrial,it,0]*np.sqrt(N),eiglvec_series[ktrial,it,:NE,1]*eigvchn_series[ktrial,it,1]*np.sqrt(N),color='tab:red',s=10,label='left eigenvector '+r'$E$',alpha=0.25)
        ax[i].scatter(eiglvec_series[ktrial,it,NE:,0]*eigvchn_series[ktrial,it,0]*np.sqrt(N),eiglvec_series[ktrial,it,NE:,1]*eigvchn_series[ktrial,it,1]*np.sqrt(N),color='tab:blue',s=10,label='left eigenvector '+r'$I$',alpha=0.25)
        
        # ax[i].scatter(eiglvec_series_rec[ktrial,it,:NE,0],eiglvec_series_rec[ktrial,it,:NE,1],color='tab:red',s=10,label='left eigenvector '+r'$E$',alpha=0.25)
        # ax[i].scatter(eiglvec_series_rec[ktrial,it,NE:,0],eiglvec_series_rec[ktrial,it,NE:,1],color='tab:blue',s=10,label='left eigenvector '+r'$I$',alpha=0.25)
        ### equal axis
        ax[i].set_aspect('equal')
        ax[i].set_xlim(xlims)
        ax[i].set_ylim(ylims)
        ax[i].set_xlabel('Random component rank 1',fontsize=14)
        ax[i].set_ylabel('Random component rank 2',fontsize=14)
        ax[i].legend()
        ### remove the top and right axes
        ax[i].spines['top'].set_color('none')
        ax[i].spines['right'].set_color('none')
        ### move the x-axis and y-axis to the center
        ax[i].spines['bottom'].set_position(('data',0))
        ax[i].spines['left'].set_position(('data',0))
        ax[i].set_title('Left eigenvector '+r'$\tau=$'+str(tau_series[it]),fontsize=14)
        ax[i].set_xticks([xlims[0],0,xlims[-1]])
        ax[i].set_yticks([xlims[0],0,xlims[-1]])
    fig.tight_layout()


In [59]:
slope_eiglvec     = np.zeros((len(idx_eff),ntau,2)) ### population
slope_eiglvec_rec = np.zeros((len(idx_eff),ntau,2))  ### population
for ik, ktrial in enumerate(idx_eff):
    for it in range(ntau):
        slope_eiglvec[ik,it,0]     = linregress(eiglvec_series[ktrial,it,:NE,0]*eigvchn_series[ktrial,it,0].real,eiglvec_series[ktrial,it,:NE,1]*eigvchn_series[ktrial,it,1].real)[0]
        slope_eiglvec_rec[ik,it,0] = linregress(eiglvec_series_rec[ktrial,it,:NE,0],eiglvec_series_rec[ktrial,it,:NE,1])[0]
        
        slope_eiglvec[ik,it,1]     = linregress(eiglvec_series[ktrial,it,NE:,0]*eigvchn_series[ktrial,it,0].real,eiglvec_series[ktrial,it,NE:,1]*eigvchn_series[ktrial,it,1].real)[0]
        slope_eiglvec_rec[ik,it,1] = linregress(eiglvec_series_rec[ktrial,it,NE:,0],eiglvec_series_rec[ktrial,it,NE:,1])[0]
        
        if np.abs(slope_eiglvec[ik,it,0])>0.82:
            slope_eiglvec[ik,it,0] = np.nan
        if np.abs(slope_eiglvec[ik,it,1])>0.82:
            slope_eiglvec[ik,it,1] = np.nan
        if np.abs(slope_eiglvec_rec[ik,it,0])>0.82:
            slope_eiglvec_rec[ik,it,0] = np.nan
        if np.abs(slope_eiglvec_rec[ik,it,1])>0.82:
            slope_eiglvec_rec[ik,it,1] = np.nan
            
### plot the (trial-mean) slope of the right eigenvectors, change with tau
fig, ax = plt.subplots(figsize=(4,2))
ax.plot(tau_series,np.nanmean(slope_eiglvec[:,:,0],axis=0),color='black',linewidth=1.5,linestyle='--',label='numerical',marker='o')
ax.plot(tau_series,np.nanmean(slope_eiglvec_rec[:,:,0],axis=0),color='black',linewidth=1.5,label='reconstructed',marker='o')

# ax.plot(tau_series,np.nanmean(slope_eiglvec[:,:,1],axis=0),color='gray',linewidth=1.5,linestyle='--',label='numerical')
# ax.plot(tau_series,np.nanmean(slope_eiglvec_rec[:,:,1],axis=0),color='gray',linewidth=1.5,label='reconstructed')

ax.set_xlim(tau_series[0],tau_series[its[1]])
ax.set_ylim(-0.1,1.5)
ax.set_yticks([0,1])
ax.set_xticks([tau_series[0],tau_series[its[1]]])
### move the x-axis to  the cecnter
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',tau_series[0]))
ax.set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax.set_ylabel('Slope of \nleft eigenvector',fontsize=12)
ax.legend()
# fig.tight_layout()

In [34]:
# slope_eiglvec     = np.zeros((len(idx_eff),ntau,2)) ### population
# slope_eiglvec_rec = np.zeros((len(idx_eff),ntau,2))  ### population
# for ik, ktrial in enumerate(idx_eff):
#     for it in range(ntau):
#         slope_eiglvec[ik,it,0]     = linregress(eiglvec_series[ktrial,it,:NE,0]*eigvchn_series[ktrial,it,0].real,eiglvec_series[ktrial,it,:NE,1]*eigvchn_series[ktrial,it,1].real)[0]
#         slope_eiglvec_rec[ik,it,0] = linregress(eiglvec_series_rec[ktrial,it,:NE,0],eiglvec_series_rec[ktrial,it,:NE,1])[0]
        
#         slope_eiglvec[ik,it,1]     = linregress(eiglvec_series[ktrial,it,NE:,0]*eigvchn_series[ktrial,it,0].real,eiglvec_series[ktrial,it,NE:,1]*eigvchn_series[ktrial,it,1].real)[0]
#         slope_eiglvec_rec[ik,it,1] = linregress(eiglvec_series_rec[ktrial,it,NE:,0],eiglvec_series_rec[ktrial,it,NE:,1])[0]
        
#         if np.abs(slope_eiglvec[ik,it,0])>0.82:
#             slope_eiglvec[ik,it,0] = np.nan
#         if np.abs(slope_eiglvec[ik,it,1])>0.82:
#             slope_eiglvec[ik,it,1] = np.nan
#         if np.abs(slope_eiglvec_rec[ik,it,0])>0.82:
#             slope_eiglvec_rec[ik,it,0] = np.nan
#         if np.abs(slope_eiglvec_rec[ik,it,1])>0.82:
#             slope_eiglvec_rec[ik,it,1] = np.nan
            
# ### plot the (trial-mean) slope of the right eigenvectors, change with tau
# fig, ax = plt.subplots(figsize=(4,2))
# ax.plot(tau_series,np.nanmean(slope_eiglvec[:,:,1],axis=0),color='black',linewidth=1.5,linestyle='--',label='numerical',marker='o')
# ax.plot(tau_series,np.nanmean(slope_eiglvec_rec[:,:,1],axis=0),color='black',linewidth=1.5,label='reconstructed',marker='o')

# # ax.plot(tau_series,np.nanmean(slope_eiglvec[:,:,1],axis=0),color='gray',linewidth=1.5,linestyle='--',label='numerical')
# # ax.plot(tau_series,np.nanmean(slope_eiglvec_rec[:,:,1],axis=0),color='gray',linewidth=1.5,label='reconstructed')

# ax.set_xlim(tau_series[0],tau_series[its[1]])
# ax.set_ylim(-0.1,1.5)
# ax.set_yticks([0,1])
# ax.set_xticks([tau_series[0],tau_series[its[1]]])
# ### move the x-axis to  the cecnter
# ax.spines['bottom'].set_position(('data',0))
# ### set other x-axis invisible
# ax.spines['top'].set_color('none')
# ax.spines['right'].set_color('none')
# ax.spines['left'].set_position(('data',tau_series[0]))
# ax.set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
# ax.set_ylabel('Slope of \nleft eigenvector',fontsize=12)
# ax.legend()
# # fig.tight_layout()

#### Dynamics

In [121]:
### compute the population mean and variance
firing_rateeq_mean,firing_rateeq_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
firing_rateeq_mean[:,:,0],firing_rateeq_mean[:,:,1] = np.mean(firing_rateeq[:,:,:NE],axis=2),np.mean(firing_rateeq[:,:,NE:],axis=2)
firing_rateeq_var[:,:,0],firing_rateeq_var[:,:,1] = np.var(firing_rateeq[:,:,:NE],axis=2),np.var(firing_rateeq[:,:,NE:],axis=2)

lowrank_eq_num_mean,lowrank_eq_num_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
lowrank_eq_num_mean[:,:,0],lowrank_eq_num_mean[:,:,1] = np.mean(lowrank_eq_num[:,:,:NE],axis=2),np.mean(lowrank_eq_num[:,:,NE:],axis=2)
lowrank_eq_num_var[:,:,0],lowrank_eq_num_var[:,:,1] = np.var(lowrank_eq_num[:,:,:NE],axis=2),np.var(lowrank_eq_num[:,:,NE:],axis=2)

In [122]:
frate_real = firing_rateeq_mean[:,-1,0].copy()
### sorting eigvchn_real 
idx = np.argsort(frate_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [ 1  5 15 12 22 18 32 13  7 33 10 29 35 16  9 26 30 23  4 17 27 28 24  8
  2 34 20 21 25  0] 30


In [123]:
#### filter the firing rate
epsilon = 1e-3
thresh_low = 2.0/10.0
### select trials
ineffective = []
for ktrial in range(trials):
    if eigvchn_series[ktrial,-1,0].real>eigvchn_series[ktrial,-1,1].real:
        print('in...',ktrial)
        ineffective = np.append(ineffective,ktrial)
        continue 
    if np.abs(eigvchn_series[ktrial,-1,0].imag)>epsilon:
        ineffective = np.append(ineffective,ktrial)
        continue
    if np.abs(eigvchn_series[ktrial,-1,1].imag)>epsilon:
        ineffective = np.append(ineffective,ktrial)
        continue
effective_trials = np.setdiff1d(np.arange(trials),ineffective)   

cut = 3
for irr in range(2):
    for it in range(ntau):
        idxsort = np.argsort(lowrank_eq[:,it,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq[idxnan,it,0] = np.nan 
        idxsort = np.argsort(lowrank_eq[:,it,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq[idxnan,it,1] = np.nan 
        ### same for lowrank_eq_num_mean 
        idxsort = np.argsort(lowrank_eq_num_mean[:,it,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq_num_mean[idxnan,it,0] = np.nan
        idxsort = np.argsort(lowrank_eq_num_mean[:,it,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq_num_mean[idxnan,it,1] = np.nan
        
for irr in range(2):
    for ktrial in range(trials):
        for it in range(0,ntau): 
            if ktrial in ineffective:
                lowrank_eq[ktrial,it,0] = np.nan
                lowrank_eq[ktrial,it,1] = np.nan
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
                continue
            if np.abs(eigvchn_series[ktrial,it,irr].imag)>epsilon:
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
                lowrank_eq[ktrial,it,0] = np.nan
                lowrank_eq[ktrial,it,1] = np.nan
                continue
            
            ### keep the better match solutions
            if np.abs(lowrank_eq_num_mean[ktrial,it,0])>thresh_low:
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
            if np.abs(lowrank_eq_num_mean[ktrial,it,1])>thresh_low:
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
            if np.abs(lowrank_eq[ktrial,it,0])>thresh_low:
                lowrank_eq[ktrial,it,0] = np.nan
            if np.abs(lowrank_eq[ktrial,it,1])>thresh_low:
                lowrank_eq[ktrial,it,1] = np.nan
                
for it in range(ntau):
    idxsort = np.argsort(firing_rateeq_mean[:,it,0])
    idxnan = np.append(idxsort[:cut],idxsort[-cut:])
    firing_rateeq_mean[idxnan,it,0] = np.nan
    
    idxsort = np.argsort(firing_rateeq_mean[:,it,1])
    idxnan = np.append(idxsort[:cut],idxsort[-cut:])
    firing_rateeq_mean[idxnan,it,1] = np.nan

for ktrial in range(trials):
    for it in range(0,ntau):                
        if np.abs(firing_rateeq_mean[ktrial,it,0])>thresh_low:
            firing_rateeq_mean[ktrial,it,0] = np.nan
        if np.abs(firing_rateeq_mean[ktrial,it,1])>thresh_low:
            firing_rateeq_mean[ktrial,it,1] = np.nan
            
for irr in range(2):
    for it in range(ntau):
        idxsort = np.argsort(contributions_lr[:,it,irr,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr[idxnan,it,irr,0] = np.nan
        
        idxsort = np.argsort(contributions_lr[:,it,irr,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr[idxnan,it,irr,1] = np.nan
        
        idxsort = np.argsort(contributions_lr_num[:,it,irr,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr_num[idxnan,it,irr,1] = np.nan
        
        idxsort = np.argsort(contributions_lr_num[:,it,irr,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr_num[idxnan,it,irr,0] = np.nan
                
#### same for the contribution from different terms
for irr in range(2):
    for ktrial in range(trials):
        for it in range(0,ntau):
            if ktrial in ineffective:
                contributions_lr[ktrial,it,irr,0] = np.nan
                contributions_lr[ktrial,it,irr,1] = np.nan
                contributions_lr_num[ktrial,it,irr,0] = np.nan
                contributions_lr_num[ktrial,it,irr,1] = np.nan
                continue
            ### keep the better match solutions
            if np.abs(contributions_lr[ktrial,it,irr,0])>thresh_low:
                contributions_lr[ktrial,it,irr,0] = np.nan
            if np.abs(contributions_lr[ktrial,it,irr,1])>thresh_low:
                contributions_lr[ktrial,it,irr,1] = np.nan
            if np.abs(contributions_lr_num[ktrial,it,irr,0])>thresh_low:
                contributions_lr_num[ktrial,it,irr,0] = np.nan
            if np.abs(contributions_lr_num[ktrial,it,irr,1])>thresh_low:
                contributions_lr_num[ktrial,it,irr,1] = np.nan
                


In [124]:
### compute the trial averaged mean 
# eigvchn_series = data['eigvchn_series']
# ntau = np.shape(eigvchn_series)[1]
# trials = np.shape(eigvchn_series)[0]
# NE, NI = params['NE'],params['NI']

mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 1.5
ths = 1.5
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)
            
            mean_leig0vec_series[ktrial,it,ir,0] = np.mean(leig0mean_series[ktrial,it,:NE,ir])
            mean_leig0vec_series[ktrial,it,ir,1] = np.mean(leig0mean_series[ktrial,it,NE:,ir])
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6#53,54#3#55#
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        if it>=6 and ir==1:
            idxpos=np.where(mean_leigvec_series[:,it,ir,0].real>0)[0]
            mean_leigvec_series[idxpos,it,ir,0] = np.nan
            
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        if it>=6 and ir==1:
            idxneg=np.where(mean_leigvec_series[:,it,ir,1].real<0)[0]
            mean_leigvec_series[idxneg,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,0].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,1].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        
        

In [125]:
### rescaled sqrt(N)
mean_leigvec_num_series *=np.sqrt(N)
mean_reigvec_num_series *=np.sqrt(N)
mean_leigvec_series *=np.sqrt(N)
mean_reigvec_series *=np.sqrt(N)

In [126]:
### compute low-rank theoretical
contribution_theo = np.zeros((ntau,2,2))
for it in range(ntau):
    if it<2: # both are fine
        for ir in range(1):
            contribution_theo[it,ir,0] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)/np.sqrt(N)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)/np.sqrt(N)*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,0],axis=0))/np.sqrt(N)
            contribution_theo[it,ir,1] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)/np.sqrt(N)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)*Inp[0]*NI)/np.sqrt(N)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,1],axis=0))/np.sqrt(N)
    else:
        for ir in range(2):
            contribution_theo[it,ir,0] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)/np.sqrt(N)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)/np.sqrt(N)*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,0],axis=0))/np.sqrt(N)
            contribution_theo[it,ir,1] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)/np.sqrt(N)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)/np.sqrt(N)*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,1],axis=0))/np.sqrt(N)
firing_rate_lwtheo = np.ones((ntau,2))*Inp[0]
firing_rate_lwtheo +=np.sum(contribution_theo.copy(),axis=1)



In [127]:
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
# ax.plot(tau_series, firing_rate0_mean[0]*np.ones((ntau,)),linestyle='--',color='gray',linewidth=1.5,alpha=0.5)
ax.fill_between(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
# ### start with the excitatory neuron population
# ax.plot(tau_series, np.nanmean(firing_rateeq_mean[:,:,0],axis=0),marker='o',linestyle='--',color='gray',alpha=1,lw=1.5)
### plot the errorbar of firing_rateeq_mean[:,:,0]
alphass=0.95
ax.errorbar(tau_series[:],np.nanmean(firing_rateeq_mean[:,:,0].real,axis=0),yerr=np.nanstd(firing_rateeq_mean[:,:,0].real,axis=0),fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
ax.plot(tau_series, firing_rate_lwtheo[:,0],marker='o',color='black',alpha=1,lw=1.5)
### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.02,0.1)
ax.set_yticks([0,0.1])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{E}$',fontsize=14)
ax.tick_params(labelsize=12)
# ax.set_title(r'$\nu_{E}$',fontsi

In [128]:
### Plot the contributions from individual ranks
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### start with the excitatory neuron population
ax.plot(tau_series, firing_rate_lwtheo[:,0],marker='o',color='black',alpha=1,lw=1.5)
### contribution from the first rank-1 component
ax.plot(tau_series, contribution_theo[:,0,0],marker='o',linestyle='--',color='gray',alpha=1,lw=1.5)
ax.plot(tau_series,contribution_theo[:,1,0],marker='o',linestyle='-',color='gray',alpha=1,lw=1.5)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.02,0.1)
ax.set_yticks([0,0.1])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)

In [101]:
firing_rateeq

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
{1 7 8 12 13 19 28 31 33 39 40 42 48 50 51 52 55 56 59 65 66 67 69 70 75 76 78 82 83 86 94}

#### reciprocal motifs

In [113]:
''' Parameters used for all networks (reciprocal and chain)'''
N =1000
J = 1/np.sqrt(N)*0.5
### get the current path location and read the data file
import os
strr= os.getcwd()
print(strr)

c = 0.4
lambda0=N*c*J 
sigma2 = N*c*(1-c)*J**2

e:\Dropbox\DailyWork\Allen_project\sonets


In [123]:

# alpharecs = ['0.800','1.600','2.400','3.200']#['0.400','0.800','1.200','1.600']#['0.100','0.200','0.300','0.400']
# alpharecs_arr = np.array([0.8,1.6,2.4,3.2])#np.array([0.4,0.8,1.2,1.6])#np.array([0.1,0.2,0.3,0.4])

alpharecs = ['0.200','0.500','0.800','1.000','1.200']#['0.400','0.800','1.200','1.600']#['0.100','0.200','0.300','0.400']
alpharecs_arr = np.array([0.2,0.5,0.8,1.0,1.2])#np.array([0.4,0.8,1.2,1.6])#np.array([0.1,0.2,0.3,0.4])

eigvrec_series = np.zeros((len(alpharecs),N),dtype=complex)

taurecs_arr = alpharecs_arr*c**2/c/(1-c)
for idxalpha, alpharec in enumerate(alpharecs):
    file_path = strr+'\\data\\w_1000_0.400_'+alpharec+'_0.000_0.000_0.000.dat'#0.200_'+alpharec+'_0.000_0.000_0.000.dat'
    Jmat = np.zeros((1000,1000))
    count =0
    with open(file_path, 'r') as file:
        content = file.read()
        if content == '0':
            print('File is empty') 
        else:
            for line in content.split('\n'):
                if line:
                    Jmat[count,:]=([(x) for x in line.split()])
                    count +=1
    Zmat2 = (Jmat-c)@(Jmat-c)
    # chainsum = 0
    # for i in range(1000):
    #     for j in range(1000):
    #         if i!=j:
    #             chainsum += Zmat2[i,j]
    Jmat_weight = Jmat*J  
    ### compute the eigenvalues and eigenvectors of the matrix
    eigenValues, eigenVectors = la.eig(Jmat_weight)
    eigvrec_series[idxalpha,:] = eigenValues

In [124]:
eigvrec_theo_series = np.zeros((len(alpharecs),2))
for idxtau, tau in enumerate(taurecs_arr):
    eigvrec_theo = np.zeros(2)
    eigvrec_theo[0]=lambda0+np.sqrt(lambda0**2+4*sigma2*tau)
    eigvrec_theo[1]=lambda0-np.sqrt(lambda0**2+4*sigma2*tau)
    eigvrec_theo /= 2
    eigvrec_theo_series[idxtau,:] = eigvrec_theo

In [125]:
eigvrec_series[:,:3]

array([[ 6.32277912+0.j        ,  0.275543  +0.j        ,
         0.26928685+0.02305987j],
       [ 6.3122946 +0.j        , -0.33869304+0.j        ,
         0.31788011+0.j        ],
       [ 6.33028728+0.j        ,  0.36227994+0.01450881j,
         0.36227994-0.01450881j],
       [ 6.33378902+0.j        , -0.41049448+0.j        ,
        -0.40709132+0.j        ],
       [ 6.32799581+0.j        ,  0.43540438+0.j        ,
         0.42390811+0.j        ]])

In [126]:
eigvrec_theo_series

array([[ 6.32581998e+00, -1.26465818e-03],
       [ 6.32771602e+00, -3.16069810e-03],
       [ 6.32961092e+00, -5.05560300e-03],
       [ 6.33087356e+00, -6.31824338e-03],
       [ 6.33213570e+00, -7.58038082e-03]])

In [127]:
### compare the eigenvalues
fig,ax = plt.subplots(figsize=(6,6))
ax.scatter(eigvrec_series[:,0].real,eigvrec_theo_series[:,0].real,marker='o',s=10,c='r',label='Simulation')
### set x==y axis     
ax.plot([0,1],[0,1],transform=ax.transAxes,ls='--',c='gray')
### the second eigenvalues   
# ax.scatter(eigvchn_series[:,1].real,eigvchn_theo_series[:,1].real,marker='o',s=10,c='b',label='Simulation')
ax.set_xlabel('Simulation')
ax.set_ylabel('Theory')
ax.legend()
plt.show()


In [128]:
taurecs_arr

array([0.13333333, 0.33333333, 0.53333333, 0.66666667, 0.8       ])

In [129]:
fig,ax = plt.subplots(figsize=(4,4))
ax.scatter(eigvrec_series[-1,1:].real,eigvrec_series[-1,1:].imag)
ax.set_aspect('equal')
ax.set_xlim()

(-0.4928201306018328, 0.4796055488844085)